In [ ]:
%pip install python-dotenv asyncio_throttle aiohttp

In [2]:
import aiohttp
import asyncio
from asyncio_throttle import Throttler
import pandas as pd
import os

In [3]:
%load_ext dotenv
%dotenv keys.env

In [6]:
api_key = os.getenv('electronicKey') # get the api key from the .env file

headers = { 
            'Content-Type' : 'application/json',
            'Authorization' : f"apikey {api_key}",
            'accept' : 'application/json'
            } # set the headers for the API call

In [12]:
df = pd.read_excel('.xlsx', dtype=str) # name of the excel file in the home directory. Columns must include: Portfolio Id, Portfolio Static URL, Service Id

In [13]:
async def update_portfolio(session, url, headers, portfolio_id): 
    async with session.get(url) as response: # get the portfolio
        content_type = response.headers.get('content-type', '')
        if 'application/json' in content_type: # check if the response is json
            data = await response.json() # convert the response to json
            try:
                data['linking_details']['static_url'] = data['linking_details']['static_url'].replace('', '') # .replace('string to replace', 'string to replace with')
                # data['linking_details']['static_url_override'] = data['linking_details']['static_url_override'].replace('', '') # uncomment to use the static_url_override field

                async with session.put(url, json=data, headers=headers) as update_response:  # update the portfolio
                    response_text = await update_response.text() # get the response text
                    print(f'Portfolio: {portfolio_id}, {response_text}') # print the response
                    with open('log.txt', 'a') as log: # log the response
                        log.write(f'Portfolio: {portfolio_id}, success \n') 
            except KeyError as error:
                with open('error_log.txt', 'a') as f:
                    f.write(f'Portfolio: {portfolio_id}, error: {error} \n')     
                    print(f'Portfolio: {portfolio_id}, error: {error}')
        else:
            response_text = await response.text()
            with open('error_log.txt', 'a') as f: # log the response
                f.write(f'Unexpected MIME type: {portfolio_id}, {response_text} \n')

In [14]:
async def process_rows(rows, headers, api_key):

    async with aiohttp.ClientSession(headers=headers) as session: # create the session
        throttler = Throttler(rate_limit=20) # set the rate limit
        tasks = []
        for index, row in rows.iterrows(): # iterate through the rows of the dataframe
            collection_id = row['Electronic Collection Id']
            portfolio_id = row['Portfolio Id']
            service_id = row['Service Id']

            url = f'https://api-na.hosted.exlibrisgroup.com/almaws/v1/electronic/e-collections/{collection_id}/e-services/{service_id}/portfolios/{portfolio_id}' # url for the API call

            async with throttler: # throttle the API calls
                tasks.append(asyncio.ensure_future(update_portfolio(session, url, headers, portfolio_id))) # add the task to the list of tasks

        await asyncio.gather(*tasks)  # gather the tasks

In [ ]:
await process_rows(df, headers, api_key)